In [1]:
from numpy.random import choice 
from bs4 import BeautifulSoup
from urllib import request
from itertools import count
from collections import OrderedDict
import re

In [ ]:
'http://korean.visitkorea.or.kr/kor/bz15/where/festival/festival.jsp?areaCode={}&year={}&month={}&gotoPage={}&cid={}'

In [ ]:
url = 'http://korean.visitkorea.or.kr/kor/bz15/where/festival/festival.jsp?cid=2376250&type=&gotoPage=&areaCode=1&year=2018&month=04'

In [2]:
def get_cid(url):
    with request.urlopen(url) as fp:
        source = fp.read()
    tags =  BeautifulSoup(source, 'html.parser').find('div', class_='news_list').find_all('a')
    return [re.search('cid=([0-9]+)', tag.attrs['href']).group(1) for tag in tags]
    return tags

In [3]:
def get_festival(url):
    festival = OrderedDict()
    with request.urlopen(url) as fp:
        source = fp.read()
    soup = BeautifulSoup(source, 'html.parser')
    
    festival['행사명'] = soup.find(id='content_title').string.strip()
    for li in soup.find('figcaption').find_all('li'):
        key = li.find('b').string.strip()
        value = ('\n'.join([c.string.strip() for c in li.find('span').children if c.string])).strip()
        festival[key] = value
            
    for tag in soup.find('ul', class_='ptList').find_all('li'):
        em = tag.find('em')
        if not em:
            break
        key = em.string.strip()
        value = '\n'.join([c.string.strip() for c in tag.children if c.string])
        festival[key] = value

    return festival

In [4]:
def scrap(areaCode='1', year='2018', month='04'):
    url = f'http://korean.visitkorea.or.kr/kor/bz15/where/festival/festival.jsp?areaCode={areaCode}&year={year}&month={month}'
    festivals = []
    for page in count(1):
        url_page = f'{url}&gotoPage={page}'
        cids = get_cid(url_page)
        if not cids:
            break
        for cid in cids:
            festivals.append(get_festival(f'{url_page}&cid={cid}'))
    return festivals

In [14]:
a = scrap('1', '2018', '04')

In [32]:
for fv in a:
    for k, v in fv.items():
        if k == '주최/주관' or k == '정보제공자' or k=='프로그램':
            continue
        v = v.replace('행사소개', '')
        v = v.replace('행사내용', '')
        v = v.replace('출연', '')
        v = v.replace('약력', '')
        print(k, ':', v)
        fname = '기러기\\' + fv['행사명'] + '.txt'
        with open(fname, 'a', encoding='utf-8') as f:
            vv = k + ':' + v + '\n'
            f.write(vv)
    print('----------------------------------')
    print('----------------------------------')

행사명 : 서울 디저트페어-벚꽃&녹차전 2018
행사기간 : 2018.04.21 ~ 2018.04.22
위치 : 서울특별시 강남구 남부순환로 3104
행사장소 : 학여울 SETEC 전시장 (학여울역 1번 출구)
연 락 처 : 02-6010-8941
홈페이지 : 북팔코믹스
http://shop.bookpalcomics.com
행사소개 : 
국내 최대 수제 디저트페어 “서울디저트페어”
16회 서울디저트페어 [벚꽃&녹차전]에서는 봄을 맞이해 벚꽃 디자인을 활용한 아트 마카롱과 아이싱 쿠키, 녹차 머랭쿠키, 말차 라떼 등 다양한 디저트를 선보인다.
공예 전시회 [서울금손페스티벌] 과 반려동물 간식 / 소품전 [고양이디저트페어] 공동 개최로 풍부해진 볼거리를 선사한다.
베이킹 소상공인과 디저트를 사랑하는 베이커리학과 학생들이 모여 매 회 독특한 컨셉으로 수제 과자전을 만들어간다.
행사내용 : 
- 1관 : 공예 소품 전시회 [서울금손페스티벌]
- 2관 : 반려동물 간식/소품전 [고양이디저트페어]
- 3관 : 수제 디저트 페스티벌 [서울디저트페어]
부대행사 : 부대행사
현장에서 상품을 받을 수 있는 다양한 이벤트 / 다양한 볼거리, 즐길거리, 먹을거리
----------------------------------
----------------------------------
행사명 : 10주년 기념 서울국제음악제 2018
행사기간 : 2018.04.24 ~ 2018.11.11
위치 : 서울특별시 서초구 남부순환로 2406
행사장소 : 에술의전당, 롯데콘서트홀 등
연 락 처 : 1544-5142
홈페이지 : http://simf.kr
행사소개 : 
서울국제음악제(Seoul International Music Festival, SIMF)는 세계적으로 인정받는 음악 단체 및 연주자들을 초청하여 국민들에게 세계 클래식 음악 문화의 조류를 알리는 역할을 톡톡하게 해내고 있다.
한편 서울국제음악제는 클래식 향유층의 확대와 전문음악인 발굴을 위한 노력의 일환으로 2018년부터 "NE

OSError: [Errno 22] Invalid argument: '기러기\\평창문화올림픽 한식문화행사 "잔치: 맛과 멋으로 따스함을 나누다" 2018.txt'

In [19]:
b = a[0]

In [20]:
b.get('행사명')

'서울 디저트페어-벚꽃&녹차전 2018'

In [33]:
for fv in a:
    fname = '기러기\\' + fv.get('행사명', '').replace('"', '') + '.txt'
    print(fname)
    with open(fname, 'w', encoding='utf-8') as f:
        f.write("행사명: " +  fv.get('행사명', '') + '\n\n\n')
        f.write("행사기간: "+ fv.get('행사기간', '')+ '\n\n\n')
        f.write("위치: " + fv.get('위치', '')+ '\n')
        f.write("행사장소: "+ fv.get('행사장소', '')+ '\n\n\n')
        f.write("연 락 처: "+ fv.get('연 락 처', '')+ '\n\n\n')
        f.write("홈페이지: "+ fv.get('홈페이지', '')+ '\n\n\n')
        f.write('행사소개:'+ fv.get('행사소개', '').replace('행사소개', '')+ '\n\n\n')
        f.write('행사내용:'+ fv.get('행사내용', '').replace('행사내용', '')+ '\n\n\n')

기러기\서울 디저트페어-벚꽃&녹차전 2018.txt
기러기\10주년 기념 서울국제음악제 2018.txt
기러기\서울 명산트레킹 2018.txt
기러기\서울문화사 30주년 기념 'SPRING WALKING FESTIVAL' 2018.txt
기러기\언더스탠드에비뉴 Young Creator's Market_마주치장 2018.txt
기러기\영등포여의도봄꽃축제 2018.txt
기러기\시끌벅적 과학축제 2018.txt
기러기\현충원, 호국의 봄을 열다 2018.txt
기러기\안산자락길 벚꽃음악회 2018.txt
기러기\생생문화재 양화진 뱃길탐방 2018.txt
기러기\한마음 치매극복 전국걷기행사 2018.txt
기러기\동대문 봄꽃축제 2018.txt
기러기\양재천 벚꽃 등(燈) 축제 2018.txt
기러기\궁중문화축전 2018.txt
기러기\4·19혁명국민문화제 2018.txt
기러기\응봉산 개나리축제 2018.txt
기러기\서울 밤도깨비 야시장 2018.txt
기러기\국회개방행사 국회의봄 2018.txt
기러기\서울국제유아교육전 2018.txt
기러기\광화문국제아트페스티벌 2018.txt
기러기\금천 하모니 벚꽃축제 2018.txt
기러기\창경궁 야간 고궁음악회 2018.txt
기러기\Lifeplus 벚꽃피크닉페스티벌 2018.txt
기러기\종로의 이야기꾼, 전기수!.txt
기러기\평창문화올림픽 한식문화행사 잔치: 맛과 멋으로 따스함을 나누다 2018.txt
기러기\석촌호수 벚꽃축제 2018.txt
기러기\기차를 통째로 빌렸다! 2018.txt
기러기\고종명성후 가례 재현행사 2018 (상반기).txt
기러기\보신각타종행사.txt
기러기\올림픽공원 9경 스탬프투어.txt


In [6]:
a = 'adsfaf'

In [8]:
?a.replace

In [ ]:
def getStrockCode(sosok=0):
    url = f'http://finance.naver.com/sise/sise_market_sum.nhn?sosok={sosok}&page='
    stockCode = OrderedDict()
    for no in count(1):
        with request.urlopen(url+str(no)) as fp:
            source = fp.read()
        tags = BeautifulSoup(source, 'html.parser').find_all('a', class_='tltle')
        if not tags:
            break
        for tag in tags:
            stockCode[tag.string] = tag['href'][-6:]
    return stockCode

def getStrockCode(sosok=0):
    url = f'http://finance.naver.com/sise/sise_market_sum.nhn?sosok={sosok}&page='
    stockCode = OrderedDict()
    for no in count(1):
        with request.urlopen(url+str(no)) as fp:
            source = fp.read()
        tags = BeautifulSoup(source, 'html.parser').find_all('a', class_='tltle')
        if not tags:
            break
        for tag in tags:
            stockCode[tag.string] = tag['href'][-6:]
    return stockCode